# Génération de la Frontière Efficiente de Markowitz

Ce notebook illustre l'utilisation de la méthode de scalarisation pour approximer la frontière efficiente d'un portefeuille d'actions.


In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os

from level1.functions import *

# Charger les données
df = load_datas()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Calcul des rendements logarithmiques
returns = f_returns_on_df(df)

# Calcul des paramètres pour l'optimisation
mu = f_mu_on_df(returns)  # Annualisation (252 jours boursiers)
Sigma = f_sigma_on_df(returns)  # Annualisation de la matrice de covariance
num_assets = len(mu)

# Calcul des Paramètres d'Optimisation

Les rendements logarithmiques sont calculés comme :

$  r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) $

Le vecteur des rendements moyens annualisés :

$ \mu = \frac{1}{T} \sum_{t=1}^T r_t \times 252 $

La matrice de covariance annualisée :

$ \Sigma = \frac{1}{T} \sum_{t=1}^T (r_t - \bar{r})(r_t - \bar{r})^T \times 252 $


# Méthode de résolution par scalarisation pour générer la frontière efficiente

Fonction rendement : $ F_1(w) = - (w^T \mu) $

Fonction risque : $ F_2(w) = w^T \Sigma w $

Fonction de cout de transaction : $ C(w) = \sum_{i=1}^{N} c_{prop} \cdot |w_i - w_i^{prev}| $

Fonction objectif scalarisée : $ F(w) = \lambda \cdot (w^T \Sigma w) - (1 - \lambda) \cdot (w^T \mu) $

## Use brute force to find different cardinality portfolios

In [ ]:
from itertools import combinations
from level2.cardinality_BF import optimize

number_of_shares = 2 # Nombre d'actions dans le portefeuille

# Use the df-columns of the index in possibilities
lambdas = np.linspace(0, 1, 40)  # 100 points entre 0 et 1

In [ ]:
f_y, f_v, f_w = optimize(df, number_of_shares, lambdas, os.cpu_count()-2)

In [ ]:
import pickle
with open(f'frontier_data_{number_of_shares}.pkl', 'wb') as f:
    pickle.dump((f_y, f_v, f_w), f)

In [ ]:
def pareto_front(volatility, yields):
    # trier les portefeuilles par risque croissant
    idx = np.argsort(volatility)
    risque_sorted = volatility[idx]
    yield_sorted = yields[idx]

    # liste des points sur le front
    front_idx = [idx[0]]
    best_yield = yield_sorted[0]

    # on parcourt en augmentant le risque et on garde les meilleurs rendements
    for i in range(1, len(risque_sorted)):
        if yield_sorted[i] > best_yield:
            front_idx.append(idx[i])
            best_yield = yield_sorted[i]

    return np.array(front_idx)

volatility = []
for e in f_v:
    volatility.extend(e)
volatility = np.array(volatility)

yields = []
for e in f_y:
    yields.extend(e)
yields = np.array(yields)

weights = []
for e in f_w:
    weights.extend(e)
weights = np.array(weights)

indices_front = pareto_front(volatility, yields)
plt.scatter(volatility, yields, alpha=0.3)
plt.scatter(volatility[indices_front], yields[indices_front], color='red', s=40, label='Front de Pareto')
plt.legend()
plt.show()

frontier_yields = yields[indices_front]
frontier_volatility = volatility[indices_front]
frontier_weights = weights[indices_front]


# Génération de la Frontière Efficiente

En faisant varier $ \lambda $ de 0 à 1, nous obtenons différents portefeuilles optimaux.


In [ ]:
print("Le portefeuille avec le rendement le plus élevé :")
max_return_index = np.argmax(frontier_yields)
print(f"Rendement : {frontier_yields[max_return_index]:.4f}, Volatilité : {frontier_volatility[max_return_index]:.4f}")
weights = frontier_weights[max_return_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

print("\nLe portefeuille avec le risque le plus faible :")
min_risk_index = np.argmin(frontier_volatility)
print(f"Rendement : {frontier_yields[min_risk_index]:.4f}, Volatilité : {frontier_volatility[min_risk_index]:.4f}")
weights = frontier_weights[min_risk_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

est ce que on peut pas relaxer une conteainte de cardinalité par une pénalisation L1 ? Si oui sous quelle condition ?
Oui, il est possible de relaxer une contrainte de cardinalité en utilisant une pénalisation L1, mais cela dépend de certaines conditions. La contrainte de cardinalité impose une limite sur le nombre d'actifs non nuls dans un portefeuille, ce qui est une contrainte non convexe et difficile à gérer directement dans les problèmes d'optimisation. En revanche, la pénalisation L1 (ou régularisation Lasso) encourage la sparsité dans les solutions en ajoutant une pénalité proportionnelle à la somme des valeurs absolues des coefficients (poids des actifs dans le portefeuille).